# Importing Libraries

In [20]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337)  # for reproducibility
import csv


from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils


In [25]:
os.chdir("/Users/Armin/Documents/Data_Science/Kaggle/Digit_Recognizer")
cwd=os.getcwd()
print "Working Dir Is:\n",cwd
print os.listdir(cwd+"/Data/")


Working Dir Is:
/Users/Armin/Documents/Data_Science/Kaggle/Digit_Recognizer
['.DS_Store', 'test.csv', 'train.csv']


In [73]:
train_data = csv.reader(open(cwd+"/Data/train.csv","rb"),delimiter=',')
x = list(train_data)
train_XY= np.array(x[1:]).astype('float')
test_data = csv.reader(open(cwd+"/Data/test.csv","rb"),delimiter=',')
x = list(test_data)
test_XY= np.array(x[1:]).astype('float')

In [74]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

In [83]:
train_percentage = 0.7
train_size = int(np.round(train_XY.shape[0]*train_percentage))

In [86]:
X_train = train_XY[0:train_size,1:]
print ("X_train size is:",X_train.shape)
y_train = train_XY[0:train_size,0]
print ("y_train size is:",y_train.shape)
X_test = train_XY[train_size:,1:   ]
print ("X_test size is:",X_test.shape)
y_test = train_XY[train_size:,0]
print ("y_test size is:",y_test.shape)

X_submission = test_XY
print ("X_submission size is:",X_submission.shape)

('X_train size is:', (29400, 784))
('y_train size is:', (29400,))
('X_test size is:', (12600, 784))
('y_test size is:', (12600,))
('X_submission size is:', (28000, 784))


In [87]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

('X_train shape:', (29400, 1, 28, 28))
(29400, 'train samples')
(12600, 'test samples')


# Setting up the figure to depict the numbers

In [ ]:
%matplotlib inline
Fig = plt.figure(figsize=(6,6))
Fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

#ax.imshow(Training_Data[1], cmap=plt.cm.binary, interpolation='nearest')
    

for i in range(64):
    ax = Fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(temp, cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    #ax.text(0, 7, str(Training_Data[i]))